<a href="https://colab.research.google.com/github/badboycherry/crops-deepLearning/blob/main/strawberry_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 셋 정리(복사)

In [ ]:
import os

# 원천 데이터셋이 있는 폴더
original_dataset_dir = '/content/drive/MyDrive/1조 공유/strawberry_data/strawberry_train'
classes_list = os.listdir(original_dataset_dir)

# train, test, val 데이터를 분리시킬 폴더
base_dir = '/content/drive/MyDrive/PROJECT/strawberry_data'
os.mkdir(base_dir)

In [ ]:
# 데이터 정리를 위한 목록 및 폴더 생성
import shutil

# base_dir 하위에 각각의 폴더 생성
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# 생성된 폴더에도 기존 데이터에 있던 하위 폴더들 생성
for cls in classes_list:
  os.mkdir(os.path.join(train_dir, cls))
  os.mkdir(os.path.join(validation_dir, cls))
  os.mkdir(os.path.join(test_dir, cls))

In [ ]:
# 데이터 확인
import math

for cls in classes_list:
  path = os.path.join(original_dataset_dir, cls) # 기존 데이터의 폴더
  fnames = os.listdir(path) # path 폴더에 있는 파일명 저장

  # 불러온 파일들을 6:2:2로 train/ val/ test로 분리
  train_size = math.floor(len(fnames)* 0.6)
  validation_size = math.floor(len(fnames)*0.2)
  test_size = math.floor(len(fnames)*0.2)

  # 각 인덱스에 해당하는 파일들을 각 폴더에 저장
  train_fnames = fnames[:train_size]
  print(f"Train size({cls}): {len(train_fnames)}")
  for fname in train_fnames:
    src = os.path.join(path, fname) # 기존 파일 경로
    dst = os.path.join(os.path.join(train_dir, cls), fname) # copy할 파일 경로
    shutil.copyfile(src, dst) # 새로 생성된 train dir에 파일 저장

  validation_fnames = fnames[train_size:(validation_size + train_size)]
  print(f"Validation size({cls}) : {len(validation_fnames)}")
  for fname in validation_fnames:
    src = os.path.join(path, fname)
    dst = os.path.join(os.path.join(validation_dir, cls), fname)
    shutil.copyfile(src,dst) # 새로 생성된 validation dir에 파일 저장

  test_fnames = fnames[(train_size + validation_size):(validation_size + train_size + test_size)]
  print(f"Test size({cls}) : {len(test_fnames)}")
  for fname in test_fnames:
    src = os.path.join(path, fname)
    dst = os.path.join(os.path.join(test_dir, cls), fname)
    shutil.copyfile(src, dst) # 새로 생성된 test dir에 파일 저장

Train size(strawberry_Graymold): 158
Validation size(strawberry_Graymold) : 52
Test size(strawberry_Graymold) : 52
Train size(strawberry_healthy): 193
Validation size(strawberry_healthy) : 64
Test size(strawberry_healthy) : 64
Train size(strawberry_mildew): 163
Validation size(strawberry_mildew) : 54
Test size(strawberry_mildew) : 54


# 데이터 정리(이동)

In [ ]:
import os
from sklearn.model_selection import train_test_split
from shutil import move

# 원천 데이터셋이 있는 폴더
original_dataset_dir = '/content/drive/MyDrive/1조 공유/strawberry_data'
classes_list = os.listdir(original_dataset_dir)

# train, test, val 데이터를 분리시킬 폴더
base_dir = '/content/drive/MyDrive/1조 공유/strawberry_data/Strawberry'
os.makedirs(base_dir, exist_ok=True)

# 데이터 정리를 위한 목록 및 폴더 생성
for subset in ['train', 'val', 'test']:
    os.makedirs(os.path.join(base_dir, subset), exist_ok=True)
    for cls in classes_list:
        os.makedirs(os.path.join(base_dir, subset, cls), exist_ok=True)

# 데이터 분리 및 이동
for cls in classes_list:
    path = os.path.join(original_dataset_dir, cls)
    fnames = os.listdir(path)

    # train:test:val = 6:2:2 로 분리
    train_fnames, testval_fnames = train_test_split(fnames, test_size=0.4, random_state=42)
    test_fnames, val_fnames = train_test_split(testval_fnames, test_size=0.5, random_state=42)

    # train 데이터 이동
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(base_dir, 'train', cls, fname)
        move(src, dst)

    # val 데이터 이동
    for fname in val_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(base_dir, 'val', cls, fname)
        move(src, dst)

    # test 데이터 이동
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(base_dir, 'test', cls, fname)
        move(src, dst)

print("데이터 분리 및 이동 완료")

데이터 분리 및 이동 완료


# YOLO TEST

- ultralytics : Ultralytics는 객체 검출(Object Detection), 세그멘테이션(Segmentation), 분류(Classification) 등과 같은 작업을 수행하기 위한 코드와 도구를 제공
- torch는 PyTorch 라이브러리로, 딥 러닝 작업을 위한 인기 있는 오픈 소스 라이브러리입니다. GPU 상에서 텐서 계산, 자동 미분, 신경망 레이어 등을 제공
- YOLO 알고리즘을 활용해 물체 검출 작업을 수행

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.2/663.2 kB 4.1 MB/s eta 0:00:00


In [ ]:
import torch
from ultralytics import YOLO

In [ ]:
# CUDA(Compute Unified Device Architecture)가 사용 가능한지 확인하는 코드
torch.cuda.is_available()

True

In [ ]:
# YOLO (You Only Look Once) 알고리즘을 사용하는 객체 검출 모델을 초기화
# 'yolov8n-cls.pt': 미리 학습된 가중치 파일의 경로/ cls (분류)
model = YOLO('yolov8n-cls.pt')

100%|██████████| 5.28M/5.28M [00:00<00:00, 66.3MB/s]


In [ ]:
# PyTorch에서 CUDA(Compute Unified Device Architecture)를 사용하여 GPU 가속을 활성화하는 부분
if torch.cuda.is_available():
  model.to('cuda')

# 모델 학습

In [ ]:
# train:모델 학습 메소드
# project : 학습된 모델과 결과물을 저장할 프로젝트 디렉토리 경로
# imgsz : 이미지 사이
# 이미지 복사하거나 하면 경고 뜸
results = model.train(data='/content/drive/MyDrive/strawberry_data/Strawberry', epochs=5, imgsz=640, project='/content/drive/MyDrive')

engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/drive/MyDrive/strawberry_data/Strawberry, epochs=5, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=/content/drive/MyDrive, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=Fal

100%|██████████| 6.23M/6.23M [00:00<00:00, 85.6MB/s]


WARNING ⚠️ NMS time limit 0.550s exceeded
AMP: checks passed ✅


train: Scanning /content/drive/.shortcut-targets-by-id/1aI-m4PcP9UcMUHMnkioIWxZj79MqZP_G/strawberry_data/Strawberry/train... 2098 images, 0 corrupt: 100%|██████████| 2098/2098 [13:37<00:00,  2.57it/s]


train: New cache created: /content/drive/.shortcut-targets-by-id/1aI-m4PcP9UcMUHMnkioIWxZj79MqZP_G/strawberry_data/Strawberry/train.cache
albumentations: RandomResizedCrop(p=1.0, height=640, width=640, scale=(0.5, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=1), HorizontalFlip(p=0.5), ColorJitter(p=0.5, brightness=[0.6, 1.4], contrast=[0.6, 1.4], saturation=[0.30000000000000004, 1.7], hue=[-0.015, 0.015]), Normalize(p=1.0, mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0), max_pixel_value=255.0), ToTensorV2(always_apply=True, p=1.0, transpose_mask=False)


...

               classes   top1_acc   top5_acc: 100%|██████████| 41/41 [09:21<00:00, 13.68s/it]

                   all      0.178          1



5 epochs completed in 2.300 hours.
Optimizer stripped from /content/drive/MyDrive/train2/weights/last.pt, 3.0MB
Optimizer stripped from /content/drive/MyDrive/train2/weights/best.pt, 3.0MB

Validating /content/drive/MyDrive/train2/weights/best.pt...
Ultralytics YOLOv8.0.229 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-cls summary (fused): 73 layers, 1438723 parameters, 0 gradients, 3.3 GFLOPs
train: /content/drive/.shortcut-targets-by-id/1aI-m4PcP9UcMUHMnkioIWxZj79MqZP_G/strawberry_data/Strawberry/train... found 3583 images in 11 classes: ERROR ❌️ requires 3 classes, not 11
val: /content/drive/.shortcut-targets-by-id/1aI-m4PcP9UcMUHMnkioIWxZj79MqZP_G/strawberry_data/Strawberry/val... found 1537 images in 7 classes: ERROR ❌️ requires 3 classes, not 7
test: /content/drive/.shortcut-targets-by-id/1aI-m4PcP9UcMUHMnkioIWxZj79MqZP_G/strawberry_data/Strawberry/test... found 2599 images in 7 classes: ERROR ❌️ requires 3 classes, not 7


               classes   top1_acc   top5_acc: 100%|██████████| 41/41 [15:58<00:00, 23.37s/it]


                   all      0.266          1
Speed: 0.6ms preprocess, 1.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/train2
Results saved to /content/drive/MyDrive/train2


# 모델 예측

In [ ]:
# 훈련된 YOLO 모델을 로드합니다. 'best.pt' 파일은 가장 좋은 성능을 보인 모델의 가중치를 나타냄
trained_model = YOLO('/content/drive/MyDrive/PROJECT/train5/weights/best.pt')

In [ ]:
# 훈련된 모델을 사용하여 이미지에서 물체를 예측
trained_model.predict('/content/drive/MyDrive/900868_20211110_1_0_0_1_2_13_0_201.jpg')


image 1/1 /content/drive/MyDrive/900868_20211110_1_0_0_1_2_13_0_201.jpg: 640x640 strawberry_healthy 0.96, strawberry_mildew 0.04, strawberry_Graymold 0.00, 4.9ms
Speed: 9.1ms preprocess, 4.9ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: None
 keypoints: None
 masks: None
 names: {0: 'strawberry_Graymold', 1: 'strawberry_healthy', 2: 'strawberry_mildew'}
 orig_img: array([[[230, 225, 222],
         [230, 225, 222],
         [232, 227, 224],
         ...,
         [234, 224, 207],
         [234, 224, 207],
         [234, 224, 207]],
 
        [[230, 225, 222],
         [230, 225, 222],
         [231, 226, 223],
         ...,
         [233, 223, 206],
         [233, 223, 206],
         [233, 223, 206]],
 
        [[230, 225, 222],
         [230, 225, 222],
         [231, 226, 223],
         ...,
         [233, 223, 206],
         [233, 223, 206],
         [233, 223, 206]],
 
        ...,
 
        [[ 86, 121,  94],
         [ 88, 123,  96],
         [ 88, 123,  96],
         ...,
         [175, 176, 174],
         [175, 176, 174],
         [174, 175, 173]],
 
        [[ 85, 120,  93],
         [ 88, 123,  96],
         [ 89, 124,  97],
         ...,
  

In [ ]:
# 'ClassifyMetrics' 객체에서 결과 딕셔너리 가져오기
results_dict = results.results_dict

# 클래스 이름과 확률을 출력
for key, value in results_dict.items(): # 결과 딕셔너리의 각 항목에 대해 반복
    if 'top1' in key:  # top-1 클래스의 경우
        top1_class = key.split('_')[-1]  # 클래스 이름 추출
        top1_probability = value # 딕셔너리의 값은 해당 클래스의 확률
        print(f'Top-1 Class: {top1_class}') # 추출한 Top-1 클래스 이름을 출력
        print(f'Top-1 Probability: {top1_probability}') # 추출한 Top-1 클래스의 확률을 출력

AttributeError: ignored

# 모델 예측

In [ ]:
trained_model.predict('/content/drive/MyDrive/3.jpg')


image 1/1 /content/drive/MyDrive/3.jpg: 640x640 strawberry_mildew 0.84, strawberry_Graymold 0.12, strawberry_healthy 0.04, 53.9ms
Speed: 2.8ms preprocess, 53.9ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: None
 keypoints: None
 masks: None
 names: {0: 'strawberry_Graymold', 1: 'strawberry_healthy', 2: 'strawberry_mildew'}
 orig_img: array([[[ 17,  17,  17],
         [ 24,  24,  24],
         [ 20,  20,  20],
         ...,
         [ 57, 122,  60],
         [ 56, 121,  59],
         [ 59, 123,  63]],
 
        [[ 21,  21,  21],
         [ 25,  25,  25],
         [ 20,  20,  20],
         ...,
         [ 54, 119,  57],
         [ 54, 119,  57],
         [ 58, 122,  62]],
 
        [[ 26,  26,  26],
         [ 26,  26,  26],
         [ 20,  20,  20],
         ...,
         [ 55, 120,  58],
         [ 56, 121,  59],
         [ 60, 124,  64]],
 
        ...,
 
        [[ 69,  60, 127],
         [ 59,  64, 125],
         [ 78,  96, 149],
         ...,
         [ 14,  13,  15],
         [ 14,  16,  17],
         [ 52,  53,  57]],
 
        [[ 66,  68,  98],
         [ 66,  61, 130],
         [ 60,  54, 136],
         ...,
  

In [ ]:
# 'ClassifyMetrics' 객체에서 결과 딕셔너리 가져오기
results_dict = results.results_dict

# 클래스 이름과 확률을 출력
for key, value in results_dict.items():
    if 'top1' in key:  # top-1 클래스의 경우
        top1_class = key.split('_')[-1]  # 클래스 이름 추출
        top1_probability = value
        print(f'Top-1 Class: {top1_class}')
        print(f'Top-1 Probability: {top1_probability}')


Top-1 Class: top1
Top-1 Probability: 0.7352941036224365


In [ ]:
results = trained_model('/content/drive/MyDrive/3.jpg')


image 1/1 /content/drive/MyDrive/3.jpg: 640x640 strawberry_mildew 0.84, strawberry_Graymold 0.12, strawberry_healthy 0.04, 76.7ms
Speed: 3.1ms preprocess, 76.7ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)


# 질병 예측

In [ ]:
results = trained_model('/content/drive/MyDrive/1151532_20211210_1_1_a1_1_2_13_3_22.jpg') # 잿빛곰팡이병


image 1/1 /content/drive/MyDrive/1151532_20211210_1_1_a1_1_2_13_3_22.jpg: 640x640 strawberry_Graymold 0.78, strawberry_mildew 0.22, strawberry_healthy 0.00, 14.1ms
Speed: 13.2ms preprocess, 14.1ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
results = trained_model('/content/drive/MyDrive/619749_20211026_1_1_a2_3_2_12_1_76.jpg') # 흰가루병


image 1/1 /content/drive/MyDrive/619749_20211026_1_1_a2_3_2_12_1_76.jpg: 640x640 strawberry_mildew 1.00, strawberry_Graymold 0.00, strawberry_healthy 0.00, 16.5ms
Speed: 18.6ms preprocess, 16.5ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
results = trained_model('/content/drive/MyDrive/620081_20211026_1_1_a2_3_2_12_1_116.jpg') # 흰가루병


image 1/1 /content/drive/MyDrive/620081_20211026_1_1_a2_3_2_12_1_116.jpg: 640x640 strawberry_mildew 1.00, strawberry_Graymold 0.00, strawberry_healthy 0.00, 4.1ms
Speed: 7.1ms preprocess, 4.1ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
results = trained_model('/content/drive/MyDrive/900870_20211110_1_0_0_1_2_13_0_203.jpg') # 정상과일


image 1/1 /content/drive/MyDrive/900870_20211110_1_0_0_1_2_13_0_203.jpg: 640x640 strawberry_healthy 1.00, strawberry_Graymold 0.00, strawberry_mildew 0.00, 4.0ms
Speed: 12.2ms preprocess, 4.0ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
results = trained_model('/content/drive/MyDrive/694432_20211102_1_0_0_3_2_12_0_720.jpg') # 정상 잎사귀 -> mildew로 판단함


image 1/1 /content/drive/MyDrive/694432_20211102_1_0_0_3_2_12_0_720.jpg: 640x640 strawberry_mildew 1.00, strawberry_healthy 0.00, strawberry_Graymold 0.00, 3.8ms
Speed: 7.4ms preprocess, 3.8ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)
